In [69]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Parameters
sequence_length = 10
num_samples = 300  # Number of samples
target_size = (64, 64)
batch_size = 1
num_classes = 4

In [70]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

train_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [71]:
train_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='training'  # Specify training subset
)

validation_generator = datagen.flow_from_directory(
    'Data/AugmentedAlzheimerDataset',
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    subset='validation'  # Specify validation subset
)

Found 27188 images belonging to 4 classes.
Found 6796 images belonging to 4 classes.


In [72]:
base_cnn = VGG16(weights='imagenet', include_top=False, input_shape=(*target_size, 3))
cnn_model = Sequential([
    base_cnn,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3)
])

In [73]:
train_sequences = np.zeros((num_samples, sequence_length, 256))
train_labels = []
for i in range(num_samples):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(train_generator)
        feature_vector = cnn_model.predict(img)
        train_sequences[i, j, :] = feature_vector.flatten()
    train_labels.append(label[0])

train_labels = np.array(train_labels)

val_sequences = np.zeros((num_samples // 5, sequence_length, 256))  # Validation sequences
val_labels = []
for i in range(num_samples // 5):
    sequence_labels = []
    for j in range(sequence_length):
        img, label = next(val_generator)
        feature_vector = cnn_model.predict(img)
        val_sequences[i, j, :] = feature_vector.flatten()
    val_labels.append(label[0])

val_labels = np.array(val_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

In [77]:

model = Sequential([
    LSTM(100, activation='relu', return_sequences=True, input_shape=(sequence_length, 256)),
    Dropout(0.3),
    LSTM(50, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

In [78]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [79]:
history = model.fit(
    train_sequences, train_labels,
    epochs=20,
    batch_size=16,
    validation_data=(val_sequences, val_labels),
    callbacks=[lr_scheduler]
)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2477 - loss: 1.4558 - val_accuracy: 0.3667 - val_loss: 1.3717 - learning_rate: 1.0000e-04
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2165 - loss: 1.4148 - val_accuracy: 0.2000 - val_loss: 1.3825 - learning_rate: 1.0000e-04
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2819 - loss: 1.3868 - val_accuracy: 0.2000 - val_loss: 1.3888 - learning_rate: 1.0000e-04
Epoch 4/20
13/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3419 - loss: 1.3496 
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3348 - loss: 1.3556 - val_accuracy: 0.2000 - val_loss: 1.3888 - learning_rate: 1.0000e-04
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3284 - loss: 1.3763 - val_accuracy: 0.2000 - val_loss: 1.3918 - learning_rate: 5.0000e-05
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3633 - loss: 1.

In [81]:
val_loss, val_accuracy = model.evaluate(val_sequences, val_labels)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

model.save("lstm_bci_model.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2062 - loss: 1.3868 
Validation Loss: 1.3911
Validation Accuracy: 0.20
